# explanation of project

Originally I was going to look at Mexican Resturants in Birmingham. However, I have now looked at them in the Houston Area. I first created a spreadsheet of neighbhorhoods and their distance from the city center. Then I created a map to show how spread out they were. Then, using forequare I created a venue list of Mexican resturants in the area finding that the most popular ones are in the city center. Not shocking when you also pull the data from the demographics seeing that the closer you are to the city center the more diverese it becomes in population and finances. Opening a middle/low end mexican resturant in the city center during this time would proove to be  sucessful venture. 

In [17]:
import numpy as np
import requests
import pandas as pd
import csv
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
import json
import matplotlib as mp
from sklearn.cluster import KMeans

print('Installed correctly')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Installed correctly


In [77]:
!pip install BeautifulSoup4


In [78]:
from bs4 import BeautifulSoup
import xml

In [267]:

#define url to webscrape
source = requests.get('https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods').text
soup = BeautifulSoup(source,'html')

#makes it easy to read
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Houston neighborhoods - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqxDDApAAEAAAF@BXx8AAABN","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Houston_neighborhoods","wgTitle":"List of Houston neighborhoods","wgCurRevisionId":947781372,"wgRevisionId":947781372,"wgArticleId":5645575,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles needing additional references from May 2018","All articles needing add

In [268]:
#table generation
table = soup.find('table',{'class':'wikitable'})
table

<table class="wikitable">
<caption>
</caption>
<tbody><tr>
<th>#
</th>
<th>Name
</th>
<th>Location relative to <a href="/wiki/Downtown_Houston" title="Downtown Houston">Downtown Houston</a>
</th>
<th>Approximate boundaries
</th></tr>
<tr>
<td>1
</td>
<td><a href="/wiki/Willowbrook,_Houston" title="Willowbrook, Houston">Willowbrook</a>
</td>
<td>Northwest
</td>
<td>Along <a href="/wiki/Texas_State_Highway_249" title="Texas State Highway 249">Texas State Highway 249</a> northwest of Beltway 8
</td></tr>
<tr>
<td>2
</td>
<td><a href="/wiki/Greenspoint,_Houston" title="Greenspoint, Houston">Greater Greenspoint</a>
</td>
<td>North
</td>
<td>Around the junction of Beltway 8 and Interstate 45 North
</td></tr>
<tr>
<td>3
</td>
<td>Carverdale
</td>
<td>Northwest
</td>
<td>South of the junction of Beltway 8 and <a href="/wiki/U.S._Route_290" title="U.S. Route 290">U.S. Route 290</a>
</td></tr>
<tr>
<td>4
</td>
<td><a href="/wiki/Fairbanks,_Houston" title="Fairbanks, Houston">Fairbanks</a> / Nort

In [269]:

#find the table we need
links = table.find_all('td')
links

[<td>1
 </td>,
 <td><a href="/wiki/Willowbrook,_Houston" title="Willowbrook, Houston">Willowbrook</a>
 </td>,
 <td>Northwest
 </td>,
 <td>Along <a href="/wiki/Texas_State_Highway_249" title="Texas State Highway 249">Texas State Highway 249</a> northwest of Beltway 8
 </td>,
 <td>2
 </td>,
 <td><a href="/wiki/Greenspoint,_Houston" title="Greenspoint, Houston">Greater Greenspoint</a>
 </td>,
 <td>North
 </td>,
 <td>Around the junction of Beltway 8 and Interstate 45 North
 </td>,
 <td>3
 </td>,
 <td>Carverdale
 </td>,
 <td>Northwest
 </td>,
 <td>South of the junction of Beltway 8 and <a href="/wiki/U.S._Route_290" title="U.S. Route 290">U.S. Route 290</a>
 </td>,
 <td>4
 </td>,
 <td><a href="/wiki/Fairbanks,_Houston" title="Fairbanks, Houston">Fairbanks</a> / Northwest Crossing
 </td>,
 <td>Northwest
 </td>,
 <td>Along U.S. Route 290 between Interstate 610 and Beltway 8
 </td>,
 <td>5
 </td>,
 <td><a href="/wiki/Inwood_Forest,_Houston" title="Inwood Forest, Houston">Greater Inwood</a>
 </

In [273]:

#create list of what we want from table
Number = []
Name = [] 
Relative = []

In [274]:

#scrapes values of td for only text and assigns them to respective areas
for i in range(0, len(links), 4):
    Number.append(links[i])
    Name.append(links[i+1].find(text=True).rstrip())
    Relative.append(links[i+2].find(text=True).rstrip())

In [275]:
#puts into a data frame
df_Houstonsuper = pd.DataFrame(data=[Number,Name,Relative]).transpose()
df_Houstonsuper.columns =['Number', 'Name','Location Relative to Downtown Houston']

In [276]:
#let see how it looks
df_Houstonsuper.head(60)

,Number,Name,Location Relative to Downtown Houston
0,<td>1 </td>,Willowbrook,Northwest
1,<td>2 </td>,Greater Greenspoint,North
2,<td>3 </td>,Carverdale,Northwest
3,<td>4 </td>,Fairbanks,Northwest
4,<td>5 </td>,Greater Inwood,Northwest
5,<td>6 </td>,Acres Home,Northwest
6,<td>7 </td>,Hidden Valley,North
7,<td>8 </td>,Westbranch,West
8,<td>9 </td>,Addicks / Park Ten,West
9,<td>10 </td>,Spring Branch,West


In [277]:
df_Houstonsuper.shape



(88, 3)

In [208]:
!pip install geocoder


In [209]:
import geocoder

In [211]:
def get_latlng(name):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Houston, Texas'.format(name))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [214]:
coords = [ get_latlng(name) for name in df_Houstonsuper['Name'].tolist()]


In [215]:
coords


[[29.95240000000007, -95.54462999999998],
 [29.939670000000035, -95.40747999999996],
 [29.849590000000035, -95.54244999999997],
 [29.852730000000065, -95.52418999999998],
 [29.86931000000004, -95.48042999999996],
 [29.87047000000007, -95.43535999999995],
 [29.88847000000004, -95.41459999999995],
 [29.83937000000003, -95.55360999999999],
 [29.814020000000028, -95.61618999999996],
 [29.803511744065073, -95.51588455414705],
 [29.826440000000048, -95.48876999999999],
 [33.217340000000036, -96.62706999999995],
 [29.82797000000005, -95.39369999999997],
 [29.79684000000003, -95.43403999999998],
 [29.796250000000043, -95.39486999999997],
 [29.77263000000005, -95.57091999999994],
 [29.75795000000005, -95.63096999999999],
 [29.74768000000006, -95.57423999999997],
 [29.725464067990657, -95.55103403692841],
 [29.732170000000053, -95.52336999999994],
 [29.760150000000067, -95.47670999999997],
 [29.888886889937478, -95.324499],
 [29.803750520427275, -95.47214794844355],
 [29.760780000000068, -95.369

In [216]:

#create temporty data frome
df_coords =pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [217]:

#Merge to old dataframe
df_Houstonsuper['Latitude']=df_coords['Latitude']
df_Houstonsuper['Longitude']=df_coords['Longitude']

In [218]:

#check combined data frame for lat and lng data
df_Houstonsuper

,Number,Name,Location Relative to Downtown Houston,Latitude,Longitude
0,<td>1 </td>,Willowbrook,Northwest,29.952400,-95.544630
1,<td>2 </td>,Greater Greenspoint,North,29.939670,-95.407480
2,<td>3 </td>,Carverdale,Northwest,29.849590,-95.542450
3,<td>4 </td>,Fairbanks,Northwest,29.852730,-95.524190
4,<td>5 </td>,Greater Inwood,Northwest,29.869310,-95.480430
5,<td>6 </td>,Acres Home,Northwest,29.870470,-95.435360
6,<td>7 </td>,Hidden Valley,North,29.888470,-95.414600
7,<td>8 </td>,Westbranch,West,29.839370,-95.553610
8,<td>9 </td>,Addicks / Park Ten,West,29.814020,-95.616190
9,<td>10 </td>,Spring Branch,West,29.803512,-95.515885


In [219]:
#check shape
print(df_Houstonsuper.shape)

(88, 5)


In [220]:
#save as csv file
df_Houstonsuper.to_csv("df_Houstonsuper.csv", index=False)

In [221]:
#get location of Houston Texas

address ='Houston, Texas'
geolocator = Nominatim(user_agent='my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Geographical location of Houston Texas is {},{}.'.format(latitude, longitude))

Geographical location of Houston Texas is 29.7589382,-95.3676974.


In [222]:
Houston_map = folium.Map(location=[latitude, longitude], zoom_start = 11)

for lat, lng, name in zip(df_Houstonsuper['Latitude'], df_Houstonsuper['Longitude'], df_Houstonsuper['Name']):
    label = '{}'.format(name)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = popup,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html=False).add_to(Houston_map)
    
Houston_map

In [223]:
#save as HTML
Houston_map.save ('Housto_map.html')

In [303]:
CLIENT_ID = 'Y4AFC4K0LHPDFHVHOGUQ5MYXH5Q3BQB1XK4YCXVFWCXEDY02' # your Foursquare ID
CLIENT_SECRET = 'VTNJJUSLQCR5UPOV3OGRVW1YMLGRFFDYYZK0HALCCBUC3PV0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4AFC4K0LHPDFHVHOGUQ5MYXH5Q3BQB1XK4YCXVFWCXEDY02
CLIENT_SECRET:VTNJJUSLQCR5UPOV3OGRVW1YMLGRFFDYYZK0HALCCBUC3PV0


In [306]:

radius = 2000
#Category ID corresponding to food and drink shop taken from Foursquare website (https://developer.foursquare.com/docs/resources/categories)

radius = 2000
LIMIT = 100

venues_list = []

for lat,lng, name in zip(df_Houstonsuper['Latitude'],df_Houstonsuper['Longitude'],df_Houstonsuper['Name']):
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for v in results:
        venues_list.append((
                name,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']))

KeyError: 'Latitude'

In [283]:

CLIENT_ID = 'Y4AFC4K0LHPDFHVHOGUQ5MYXH5Q3BQB1XK4YCXVFWCXEDY02' # your Foursquare ID
CLIENT_SECRET = 'VTNJJUSLQCR5UPOV3OGRVW1YMLGRFFDYYZK0HALCCBUC3PV0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4AFC4K0LHPDFHVHOGUQ5MYXH5Q3BQB1XK4YCXVFWCXEDY02
CLIENT_SECRET:VTNJJUSLQCR5UPOV3OGRVW1YMLGRFFDYYZK0HALCCBUC3PV0


In [307]:

from geopy.geocoders import Nominatim
address = 'Houston, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

29.7589382 -95.3676974


In [310]:

search_query = 'Mexican'
radius = 500
print(search_query + ' .... OK!')

Mexican .... OK!


In [311]:

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Y4AFC4K0LHPDFHVHOGUQ5MYXH5Q3BQB1XK4YCXVFWCXEDY02&client_secret=VTNJJUSLQCR5UPOV3OGRVW1YMLGRFFDYYZK0HALCCBUC3PV0&ll=29.7589382,-95.3676974&v=20180604&query=Mexican&radius=500&limit=100'

In [312]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eadec77963d29001bc6fb75'},
 'response': {'venues': [{'id': '5390eeaa498e131b9e9908a1',
    'name': 'M R Mexican Grill',
    'location': {'address': '1001 Fannin St Ste M162',
     'lat': 29.756250984167107,
     'lng': -95.36479746615205,
     'labeledLatLngs': [{'label': 'display',
       'lat': 29.756250984167107,
       'lng': -95.36479746615205}],
     'distance': 409,
     'postalCode': '77002',
     'cc': 'US',
     'city': 'Houston',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['1001 Fannin St Ste M162',
      'Houston, TX 77002',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'shortName': 'Mexican',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588456931',
    'hasPerk': False},
   {'id

In [314]:
from pandas.io.json import json_normalize

In [315]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,5390eeaa498e131b9e9908a1,1001 Fannin St Ste M162,US,Houston,United States,NaN,409,"[1001 Fannin St Ste M162, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756250984167107, 'lng': -95.36479746615205}]",29.756251,-95.364797,77002,TX,M R Mexican Grill,v-1588456931
1,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,4c65765f19f3c9b63a139dff,1000 Main St,US,Houston,United States,Tunnel Level,353,"[1000 Main St (Tunnel Level), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.75651451947814, 'lng': -95.36533952769459}]",29.756515,-95.365340,77002,TX,Poblano's Mexican Grill #2,v-1588456931
2,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,5dc4502e587ccc000857e2e9,1000 Main St Ste T33,US,Houston,United States,NaN,353,"[1000 Main St Ste T33, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756482, 'lng': -95.365377}]",29.756482,-95.365377,77002,TX,El Regio Mexican Grill,v-1588456931
3,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,53b40b88498ed3f8352e16e1,NaN,US,Houston,United States,NaN,511,"[Houston, TX, United States]","[{'label': 'display', 'lat': 29.755855, 'lng': -95.363777}]",29.755855,-95.363777,NaN,TX,m r mexican grill,v-1588456931
4,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,4a4a3e6ef964a52099ab1fe3,909 Texas St Ste B,US,Houston,United States,Texas & Main,511,"[909 Texas St Ste B (Texas & Main), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.76023602568847, 'lng': -95.36261549538631}]",29.760236,-95.362615,77002,TX,Chipotle Mexican Grill,v-1588456931


In [316]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,M R Mexican Grill,Mexican Restaurant,1001 Fannin St Ste M162,US,Houston,United States,NaN,409,"[1001 Fannin St Ste M162, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756250984167107, 'lng': -95.36479746615205}]",29.756251,-95.364797,77002,TX,5390eeaa498e131b9e9908a1
1,Poblano's Mexican Grill #2,Mexican Restaurant,1000 Main St,US,Houston,United States,Tunnel Level,353,"[1000 Main St (Tunnel Level), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.75651451947814, 'lng': -95.36533952769459}]",29.756515,-95.365340,77002,TX,4c65765f19f3c9b63a139dff
2,El Regio Mexican Grill,Mexican Restaurant,1000 Main St Ste T33,US,Houston,United States,NaN,353,"[1000 Main St Ste T33, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756482, 'lng': -95.365377}]",29.756482,-95.365377,77002,TX,5dc4502e587ccc000857e2e9
3,m r mexican grill,Mexican Restaurant,NaN,US,Houston,United States,NaN,511,"[Houston, TX, United States]","[{'label': 'display', 'lat': 29.755855, 'lng': -95.363777}]",29.755855,-95.363777,NaN,TX,53b40b88498ed3f8352e16e1
4,Chipotle Mexican Grill,Mexican Restaurant,909 Texas St Ste B,US,Houston,United States,Texas & Main,511,"[909 Texas St Ste B (Texas & Main), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.76023602568847, 'lng': -95.36261549538631}]",29.760236,-95.362615,77002,TX,4a4a3e6ef964a52099ab1fe3
5,Jefe Bar,Nightclub,502 Main St,US,Houston,United States,NaN,519,"[502 Main St, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.760602000000002, 'lng': -95.362679}, {'label': 'entrance', 'lat': 29.760577, 'lng': -95.36218}]",29.760602,-95.362679,77002,TX,4c380bf52c8020a127448a00


In [317]:
dataframe_filtered.name

0             M R Mexican Grill
1    Poblano's Mexican Grill #2
2        El Regio Mexican Grill
3             m r mexican grill
4        Chipotle Mexican Grill
5                      Jefe Bar
Name: name, dtype: object

In [318]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the City Center 

# add a red circle marker to represent the City center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [320]:
dataframe.head(200)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,5390eeaa498e131b9e9908a1,1001 Fannin St Ste M162,US,Houston,United States,NaN,409,"[1001 Fannin St Ste M162, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756250984167107, 'lng': -95.36479746615205}]",29.756251,-95.364797,77002,TX,M R Mexican Grill,v-1588456931
1,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,4c65765f19f3c9b63a139dff,1000 Main St,US,Houston,United States,Tunnel Level,353,"[1000 Main St (Tunnel Level), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.75651451947814, 'lng': -95.36533952769459}]",29.756515,-95.365340,77002,TX,Poblano's Mexican Grill #2,v-1588456931
2,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,5dc4502e587ccc000857e2e9,1000 Main St Ste T33,US,Houston,United States,NaN,353,"[1000 Main St Ste T33, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.756482, 'lng': -95.365377}]",29.756482,-95.365377,77002,TX,El Regio Mexican Grill,v-1588456931
3,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,53b40b88498ed3f8352e16e1,NaN,US,Houston,United States,NaN,511,"[Houston, TX, United States]","[{'label': 'display', 'lat': 29.755855, 'lng': -95.363777}]",29.755855,-95.363777,NaN,TX,m r mexican grill,v-1588456931
4,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",False,4a4a3e6ef964a52099ab1fe3,909 Texas St Ste B,US,Houston,United States,Texas & Main,511,"[909 Texas St Ste B (Texas & Main), Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.76023602568847, 'lng': -95.36261549538631}]",29.760236,-95.362615,77002,TX,Chipotle Mexican Grill,v-1588456931
5,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'Nightclub', 'pluralName': 'Nightclubs', 'shortName': 'Nightclub', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_', 'suffix': '.png'}, 'primary': True}]",False,4c380bf52c8020a127448a00,502 Main St,US,Houston,United States,NaN,519,"[502 Main St, Houston, TX 77002, United States]","[{'label': 'display', 'lat': 29.760602000000002, 'lng': -95.362679}, {'label': 'entrance', 'lat': 29.760577, 'lng': -95.36218}]",29.760602,-95.362679,77002,TX,Jefe Bar,v-1588456931
